In [1]:
import pandas as pd

In [2]:
weatherPred = pd.read_csv("../data/forecast_weather.csv")

In [6]:
a = weatherPred[['latitude', 'longitude']].value_counts()

In [16]:
unique_pairs = list(set(zip(weatherPred['latitude'], weatherPred[ 'longitude'])))

In [ ]:
{
  "0": "HARJUMAA",       (59.4 24.2), (59.4 24.7), (59.4 25.2), (59.4 25.7), (59.1 24.2), (59.1 25.2)
  "1": "HIIUMAA",        (58.8 22.7), ocean: (58.8 22.2),(59.1 22.7),(58.8 23.2)
  "2": "IDA-VIRUMAA",    (59.4 27.2), (59.4 27.7), (59.1 27.2), (59.1 27.7)
  "3": "JÄRVAMAA",       (58.8 25.7), (59.1 25.7)
  "4": "JÕGEVAMAA",      (58.8 26.2), (58.8 26.7), (58.5 26.2)
  "5": "LÄÄNE-VIRUMAA",  (59.4 26.2), (59.4 26.7), (59.1 26.2), (59.1 26.7)
  "6": "LÄÄNEMAA",       (59.1 23.7), (58.8 23.7)   #over boarder: (58.5 23.7)
  "7": "PÄRNUMAA",       (58.5 23.7), (58.5 24.2), (58.5 24.7), (58.2 24.7)
  "8": "PÕLVAMAA",       (58.2 27.2), (57.9 27.7)
  "9": "RAPLAMAA",       (59.1 24.7), (58.8 24.2), (58.8 25.2), (58.8 24.7)
  "10": "SAAREMAA",      (58.5 22.2), (58.5 22.7), (58.5 23.2), (58.2 22.2) ocean:(58.2 22.7),(57.9 22.2)
  "11": "TARTUMAA",      (58.2 26.2), (58.2 26.7), (58.5 27.2), (58.5 26.7)
  "12": "UNKNOWN",
  "13": "VALGAMAA",      (57.9 25.7), (57.9 26.2)
  "14": "VILJANDIMAA",   (58.5 25.2), (58.5 25.7), (58.2 25.2), (58.2 25.7)
  "15": "VÕRUMAA"        (57.9 26.7), (57.9 27.2), (57.6 26.7), (57.6 27.2)
}

In [83]:
import folium
import geopandas as gpd

# Load GeoJSON data for Estonian counties
geojson_path = 'geo-countries/data/ne_10m_admin_1_states_provinces.shp'  # Update with your GeoJSON file path
estonia_geojson = gpd.read_file(geojson_path)
estonia_geojson = estonia_geojson.loc[estonia_geojson.admin == "Estonia"]
minx, miny, maxx, maxy = estonia_geojson.geometry.total_bounds

# Create a map centered around the bounding box
mymap = folium.Map(location=[(miny + maxy) / 2, (minx + maxx) / 2], zoom_start=6)

# Add GeoJSON layer for Estonian counties
folium.GeoJson(estonia_geojson).add_to(mymap)

# Example set of geo coordinates
coordinates = unique_pairs#[(37.7749, -122.4194), (40.7128, -74.0060), (51.5074, -0.1278)]  # Add more as needed
# Add markers for each set of coordinates
for coord in coordinates:
    folium.Marker(location=[coord[0], coord[1]],
                  popup=folium.Popup(str(coord[0])+' '+str(coord[1]), parse_html=True),
                  icon=folium.Icon(color='blue')).add_to(mymap)

for idx, row in estonia_geojson.iterrows():
    county_name = row['woe_name']
    centroid = [row['geometry'].centroid.y, row['geometry'].centroid.x]

    folium.Marker(location=centroid,
                  popup=folium.Popup(county_name, parse_html=True),
                  icon=folium.Icon(color='blue')).add_to(mymap)

    #folium.Marker(coord).add_to(mymap)

# Save the map as an HTML file or display it
mymap.save('estonia.html')


In [48]:
estonia_geojson.woe_name

123       Viljandi
126          Pärnu
128          Valga
131           Võru
1604      Ida-Viru
1605         Põlva
1606         Tartu
1607        Jõgeva
2384         Lääne
2385         Harju
2386    Lääne-Viru
2919         Saare
2920          Hiiu
3739         Rapla
3740         Järva
Name: woe_name, dtype: object

In [79]:
import numpy as np
centroids_x = estonia_geojson.geometry.centroid.x.to_list()
centroids_y = estonia_geojson.geometry.centroid.y.to_list()
numCent = len(centroids_x)

coord_cluster = []
for coord in coordinates:
    minError = 1e200
    cluster = -1
    for i in range(0,numCent):
        error = np.sqrt((coord[0]-centroids_x[i])**2 + (coord[1]-centroids_y[i])**2)
        if error < minError:
            minError = error
            cluster = i
    coord_cluster.append([cluster, minError])

/var/folders/15/v1vyry112dv3d398dy47k8v40000gn/T/ipykernel_87762/799559972.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_x = estonia_geojson.geometry.centroid.x.to_list()
/var/folders/15/v1vyry112dv3d398dy47k8v40000gn/T/ipykernel_87762/799559972.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_y = estonia_geojson.geometry.centroid.y.to_list()


In [81]:
centroids_x

[25.558364059539933,
 24.550321648922793,
 26.14302669183717,
 26.91115291686715,
 27.327483563720584,
 27.162798202382948,
 26.82996683757416,
 26.536903689900708,
 23.760348418177717,
 24.924347516442776,
 26.355665774943418,
 22.537559658585536,
 22.61868018616204,
 24.70288373555266,
 25.66666660100041]

In [74]:
unique_pairs[0][0]


57.6

In [43]:
from shapely.geometry import Point

# Create a GeoDataFrame for the coordinates
geometry = [Point(lon, lat) for lon, lat in coordinates]
points_gdf = gpd.GeoDataFrame(geometry=geometry, crs=estonia_geojson.crs)

# Perform spatial join to find which shape/boundary each point is in
joined = gpd.sjoin(points_gdf, estonia_geojson, how='left', op='within')

/Users/matthiaskargl/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/matthiaskargl/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
